<a href="https://colab.research.google.com/github/gkrry2723/ML_pytorch_study/blob/master/%EC%A0%84%EC%9D%B4%ED%95%99%EC%8A%B5_%ED%83%80%EC%BD%94%EC%99%80%EB%B8%8C%EB%A6%AC%EB%98%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://github.com/lucidfrontier45/PyTorch-Book/raw/master/data/taco_and_burrito.tar.gz
!tar -zxvf taco_and_burrito.tar.gz

--2021-01-25 02:57:49--  https://github.com/lucidfrontier45/PyTorch-Book/raw/master/data/taco_and_burrito.tar.gz
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/lucidfrontier45/PyTorch-Book/master/data/taco_and_burrito.tar.gz [following]
--2021-01-25 02:57:49--  https://raw.githubusercontent.com/lucidfrontier45/PyTorch-Book/master/data/taco_and_burrito.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15292798 (15M) [application/octet-stream]
Saving to: ‘taco_and_burrito.tar.gz’

taco_and_burrito.ta 100%[===================>]  14.58M  --.-KB/s    in 0.09s   

2021-01-25 02:57:49 (167 MB/s) - ‘ta

데이터 준비

In [23]:
import torch
from torch import nn,optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset) 
from torchvision.datasets import ImageFolder
from torchvision import transforms

# 저런식으로 디렉토리 구조를 만들어두면 imagefolder로 읽어서 dataset으로 쉽게 변환할 수 있다.
# imagefolder를 사용하면 지정된 디렉터리의 하위 디렉터리 명을 분류명으로 사용한다.
# 파이토치의 imageNet을 전이학습에 사용
# imagenet을 사용한 학습완료된 모델은 224*224픽셀의 이미지를 입력 데이터로 받음.
# 그러므로 해당 크기로 크롭!

# ImageFolder 함수를 사용해서 Dataset 작성
train_imgs = ImageFolder("/content/taco_and_burrito/train/",
    transform=transforms.Compose([
        transforms.RandomCrop(224),
        transforms.ToTensor()]
))
test_imgs = ImageFolder("/content/taco_and_burrito/test/",
    transform=transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor()]
))

# DataLoader 작성
train_loader = DataLoader(
    train_imgs, batch_size=32, shuffle=True)
test_loader = DataLoader(
    test_imgs, batch_size=32, shuffle=False)

In [24]:
# 분류명과 인덱스 대응관계 확인
print(train_imgs.classes)
print(train_imgs.class_to_idx)

['burrito', 'taco']
{'burrito': 0, 'taco': 1}


전이학습

In [29]:
#사전 학습이 끝난 모델을 불러오기 -> ResNet18
#이 모델은 출력 선형 계층을 fc라는 이름으로 가져온다.
# 먼저 모든 파라미터를 미분 대상에서 제외시키고 fc에 새로운 선형계층을 설정한다.
# 이러면 마지막 선형계층만 미분 대상이 된다.

from torchvision import models

net = models.resnet18(pretrained=True)

for p in net.parameters():
  p.requires_grad = False

fc_input_dim = net.fc.in_features
net.fc = nn.Linear(fc_input_dim,2)


훈련함수

In [43]:
import tqdm

def eval_net(net,data_loader,device="cpu"):
  #test니까 Dropout, batch를 무효화
  net.eval()
  ys = []
  ypreds = []
  for x, y in data_loader:
    x=x.to(device)
    y=y.to(device)
    #확률이 가장 큰 분류를 예측
    #eval이니까 자동미분에 필요한 처리는 off로
    with torch.no_grad():
      _,y_pred=net(x).max(1)
    ys.append(y)
    ypreds.append(y_pred)
  ys = torch.cat(ys)
  ypreds = torch.cat(ypreds)

  acc = (ys==ypreds).float().sum() / len(ys)
  return acc.item()

def train_net(net, train_loader, test_loader,only_fc=True,
              optimizer_cls=optim.Adam,
              loss_fn=nn.CrossEntropyLoss(),
              n_iter=10, device="cpu"):
    train_losses = []
    train_acc = []
    val_acc = []
    if only_fc:
        # 마지막 선형 계층의 파라미터만
        # optimizer에 전달
        optimizer = optimizer_cls(net.fc.parameters())
    else:
        optimizer = optimizer_cls(net.parameters())
    for epoch in range(n_iter):
        running_loss = 0.0
        # 신경망을 훈련 모드로 설정
        net.train()
        n = 0
        n_acc = 0
        # 시간이 많이 걸리므로 tqdm을 사용해서 진행바를 표시
        for i, (xx, yy) in tqdm.tqdm(enumerate(train_loader),total=len(train_loader)):
            xx = xx.to(device)
            yy = yy.to(device)
            h = net(xx)
            loss = loss_fn(h, yy)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            n += len(xx)
            _, y_pred = h.max(1)
            n_acc += (yy == y_pred).float().sum().item()
        train_losses.append(running_loss / i)
        # 훈련 데이터의 예측 정확도
        train_acc.append(n_acc / n)

        # 검증 데이터의 예측 정확도
        val_acc.append(eval_net(net, test_loader, device))
        # epoch의 결과 표시
        print(epoch, train_losses[-1], train_acc[-1],
            val_acc[-1], flush=True)


In [44]:
net.to("cuda:0")
train_net(net, train_loader, test_loader, n_iter=10, device="cuda:0")












  0%|          | 0/23 [00:00<?, ?it/s]










  4%|▍         | 1/23 [00:00<00:02,  8.09it/s]










  9%|▊         | 2/23 [00:00<00:02,  8.16it/s]










 13%|█▎        | 3/23 [00:00<00:02,  8.18it/s]










 17%|█▋        | 4/23 [00:00<00:02,  8.29it/s]










 22%|██▏       | 5/23 [00:00<00:02,  8.33it/s]










 26%|██▌       | 6/23 [00:00<00:02,  8.44it/s]










 30%|███       | 7/23 [00:00<00:01,  8.42it/s]










 35%|███▍      | 8/23 [00:00<00:01,  8.41it/s]










 39%|███▉      | 9/23 [00:01<00:01,  8.32it/s]










 43%|████▎     | 10/23 [00:01<00:01,  8.36it/s]










 48%|████▊     | 11/23 [00:01<00:01,  8.41it/s]










 52%|█████▏    | 12/23 [00:01<00:01,  8.49it/s]










 57%|█████▋    | 13/23 [00:01<00:01,  8.42it/s]










 61%|██████    | 14/23 [00:01<00:01,  8.52it/s]










 65%|██████▌   | 15/23 [00:01<00:00,  8.46it/s]










 70%|██████▉   | 16/23 [00:01<00:00,  8.44it/s]










 74%|███████▍  | 17/2

0 0.25838166543028573 0.898876404494382 0.8833333849906921













  0%|          | 0/23 [00:00<?, ?it/s]










  4%|▍         | 1/23 [00:00<00:02,  8.48it/s]










  9%|▊         | 2/23 [00:00<00:02,  8.43it/s]










 13%|█▎        | 3/23 [00:00<00:02,  8.38it/s]










 17%|█▋        | 4/23 [00:00<00:02,  8.43it/s]










 22%|██▏       | 5/23 [00:00<00:02,  8.39it/s]










 26%|██▌       | 6/23 [00:00<00:02,  8.44it/s]










 30%|███       | 7/23 [00:00<00:01,  8.43it/s]










 35%|███▍      | 8/23 [00:00<00:01,  8.52it/s]










 39%|███▉      | 9/23 [00:01<00:01,  8.51it/s]










 43%|████▎     | 10/23 [00:01<00:01,  8.51it/s]










 48%|████▊     | 11/23 [00:01<00:01,  8.58it/s]










 52%|█████▏    | 12/23 [00:01<00:01,  8.57it/s]










 57%|█████▋    | 13/23 [00:01<00:01,  8.56it/s]










 61%|██████    | 14/23 [00:01<00:01,  8.31it/s]










 65%|██████▌   | 15/23 [00:01<00:00,  8.37it/s]










 70%|██████▉   | 16/23 [00:01<00:00,  8.40it/s]










 74%|███████▍  | 17/2

1 0.28636157512664795 0.8764044943820225 0.8333333730697632













  0%|          | 0/23 [00:00<?, ?it/s]










  4%|▍         | 1/23 [00:00<00:02,  8.81it/s]










  9%|▊         | 2/23 [00:00<00:02,  8.60it/s]










 13%|█▎        | 3/23 [00:00<00:02,  8.61it/s]










 17%|█▋        | 4/23 [00:00<00:02,  8.58it/s]










 22%|██▏       | 5/23 [00:00<00:02,  8.63it/s]










 26%|██▌       | 6/23 [00:00<00:01,  8.52it/s]










 30%|███       | 7/23 [00:00<00:01,  8.56it/s]










 35%|███▍      | 8/23 [00:00<00:01,  8.52it/s]










 39%|███▉      | 9/23 [00:01<00:01,  8.49it/s]










 43%|████▎     | 10/23 [00:01<00:01,  8.54it/s]










 48%|████▊     | 11/23 [00:01<00:01,  8.52it/s]










 52%|█████▏    | 12/23 [00:01<00:01,  8.55it/s]










 57%|█████▋    | 13/23 [00:01<00:01,  8.57it/s]










 61%|██████    | 14/23 [00:01<00:01,  8.53it/s]










 65%|██████▌   | 15/23 [00:01<00:00,  8.50it/s]










 70%|██████▉   | 16/23 [00:01<00:00,  8.49it/s]










 74%|███████▍  | 17/2

...

7 0.2558514438569546 0.9101123595505618 0.8333333730697632













  0%|          | 0/23 [00:00<?, ?it/s]










  4%|▍         | 1/23 [00:00<00:02,  8.68it/s]










  9%|▊         | 2/23 [00:00<00:02,  8.77it/s]










 13%|█▎        | 3/23 [00:00<00:02,  8.69it/s]










 17%|█▋        | 4/23 [00:00<00:02,  8.69it/s]










 22%|██▏       | 5/23 [00:00<00:02,  8.71it/s]










 26%|██▌       | 6/23 [00:00<00:01,  8.73it/s]










 30%|███       | 7/23 [00:00<00:01,  8.55it/s]










 35%|███▍      | 8/23 [00:00<00:01,  8.62it/s]










 39%|███▉      | 9/23 [00:01<00:01,  8.62it/s]










 43%|████▎     | 10/23 [00:01<00:01,  8.57it/s]










 48%|████▊     | 11/23 [00:01<00:01,  8.58it/s]










 52%|█████▏    | 12/23 [00:01<00:01,  8.54it/s]










 57%|█████▋    | 13/23 [00:01<00:01,  8.60it/s]










 61%|██████    | 14/23 [00:01<00:01,  8.70it/s]










 65%|██████▌   | 15/23 [00:01<00:00,  8.57it/s]










 70%|██████▉   | 16/23 [00:01<00:00,  8.51it/s]










 74%|███████▍  | 17/2

8 0.2774479372257536 0.8862359550561798 0.8333333730697632













  0%|          | 0/23 [00:00<?, ?it/s]










  4%|▍         | 1/23 [00:00<00:02,  8.50it/s]










  9%|▊         | 2/23 [00:00<00:02,  8.58it/s]










 13%|█▎        | 3/23 [00:00<00:02,  8.52it/s]










 17%|█▋        | 4/23 [00:00<00:02,  8.65it/s]










 22%|██▏       | 5/23 [00:00<00:02,  8.63it/s]










 26%|██▌       | 6/23 [00:00<00:01,  8.59it/s]










 30%|███       | 7/23 [00:00<00:01,  8.48it/s]










 35%|███▍      | 8/23 [00:00<00:01,  8.55it/s]










 39%|███▉      | 9/23 [00:01<00:01,  8.52it/s]










 43%|████▎     | 10/23 [00:01<00:01,  8.63it/s]










 48%|████▊     | 11/23 [00:01<00:01,  8.67it/s]










 52%|█████▏    | 12/23 [00:01<00:01,  8.51it/s]










 57%|█████▋    | 13/23 [00:01<00:01,  8.52it/s]










 61%|██████    | 14/23 [00:01<00:01,  8.49it/s]










 65%|██████▌   | 15/23 [00:01<00:00,  8.44it/s]










 70%|██████▉   | 16/23 [00:01<00:00,  8.45it/s]










 74%|███████▍  | 17/2

9 0.2779878896068443 0.8918539325842697 0.8666667342185974
